<a href="https://colab.research.google.com/github/shikha-aggarwal/wodehouse-generator/blob/main/gpt2_huggingface_GPU_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/huggingface/transformers

import os
os.chdir('/content/transformers')

# # Use language modeling version as of April 21st.
!git checkout b1ff0b2ae7d368b7db3a8a8472a29cc195d278d8

!pip install .
!pip install -r ./examples/requirements.txt
!pip install urllib3==1.25.10

### If you see errors during the above installation step, restarting the runtime might help.

In [3]:
os.chdir('/content/transformers/examples/')
# !pip install dict_to_obj

In [20]:
import torch
import run_language_modeling
import run_generation
import collections
import random
import numpy as np

from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

# Text processing
import json
from pathlib import Path
from glob import glob
import os
from concurrent.futures import ProcessPoolExecutor
from itertools import chain
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# all_data_txt = '/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/all_novels/train.txt'

# with open(all_data_txt, "w") as f:
#     f.write(training_text)
#     f.close()

### Text Preprocessing

##### Using sliding window of 8 sentences

In [21]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')
sent_tokenize = sentence_tokenizer.tokenize

def flatten(iterable):
    return chain.from_iterable(iterable)

def preprocess_book(book_txt):
    if "404 Not Found" in book_txt:
        return ""
    
    start_idx = book_txt.index("START OF THIS PROJECT GUTENBERG") + 100
    end_idx = book_txt.index("END OF THIS PROJECT") - 20
    txt =  book_txt[start_idx: end_idx]
    return re.sub("\s+", " ", txt)


def process_book(book_path):
    try:
        txt = preprocess_book(Path(book_path).read_text("utf-8"))
        sentences = [s for s in sent_tokenize(txt) if len(s) >= 16]
        windowed_sentences = []
        for snt in range(len(sentences)):
            windowed_sentences.append(" ".join(sentences[snt: snt + 8]))
        return windowed_sentences
    except:
        print(f"Could not parse \n{book_path}\n")
        return []

In [22]:
train_data_directory = '/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/all_novels/'
sliding_train_data = '/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/train_sliding.txt'

books = []
for filename in os.listdir(train_data_directory):
  file_path = os.path.join(train_data_directory, filename)
  books.append(file_path)

buffer, BUFFER_SIZE = [], 100000
with open(sliding_train_data, "w") as file:
  for i, sentence in enumerate(flatten(process_book(f) for f in books)):
    if len(buffer) >= BUFFER_SIZE:
      file.write("\n".join(buffer))
      buffer.clear()
      print(i, end="\r")
    buffer.append(sentence)
  if len(buffer) > 0:
    file.write("\n".join(buffer))
    buffer.clear()

Could not parse 
/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/all_novels/pg59254.txt



In [26]:
!head /content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/train_sliding.txt

uck Greif and the Online Distributed Proofreading Team INDISCRETIONS OF ARCHIE By P. G. Wodehouse It wasn't Archie's fault really. Its true he went to America and fell in love with Lucille, the daughter of a millionaire hotel proprietor and if he did marry her--well, what else was there to do? From his point of view, the whole thing was a thoroughly good egg; but Mr. Brewster, his father-in-law, thought differently, Archie had neither money nor occupation, which was distasteful in the eyes of the industrious Mr. Brewster; but the real bar was the fact that he had once adversely criticised one of his hotels. Archie does his best to heal the breach; but, being something of an ass, genus priceless, he finds it almost beyond his powers to placate “the man-eating fish” whom Providence has given him as a father-in-law P. G. Wodehouse AUTHOR OF “THE LITTLE WARRIOR,” “A DAMSEL IN DISTRESS,” “UNEASY MONEY,” ETC. NEW YORK GEORGE H. DORAN COMPANY COPYRIGHT,1921, BY GEORGE H, DORAN COMPANY COPYRIG

In [31]:


# run_language_modeling_with_tokenizers.py -- it's the version with support for fast tokenizers, see above
!python run_language_modeling.py \
    --train_data_file=/content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/train_sliding.txt \
    --eval_data_file=/content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/all_novels/validate.txt \
    --output_dir='/content/drive/My Drive/finetuned_models/sliding/wodehouse' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --mlm \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --line_by_line \
    --learning_rate=1e-5 \
    --num_train_epochs=5 \
    --save_total_limit=2 \
    --save_steps=5000 \
    --per_gpu_train_batch_size=8 \
    --warmup_steps=10000 \
    --logging_steps=500 \
    --gradient_accumulation_steps=4 \
    --seed=666 --block_size=512

2020-10-21 23:57:28.057317: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
10/21/2020 23:57:29 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
10/21/2020 23:57:29 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-config.json from cache at /root/.cache/torch/transformers/98aa65385e18b0efd17acd8bf64dcdf21406bb0c99c801c2d3c9f6bfd1f48f29.250d6dc755ccb17d19c7c1a7677636683aa35f0f6cb5461b3c0587bc091551a0
10/21/2020 23:57:29 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,

In [6]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/finetuned_models/wodehouse' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --save_steps=1500 \
    --train_data_file=/content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/train_sliding.txt \
    --do_eval \
    --eval_data_file=/content/drive/My\ Drive/Colab\ Notebooks/wodehouse_generator/data/all_novels/validate.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=128 \
    --gradient_accumulation_steps=5 \
    --overwrite_output_dir # too lazy to delete previous failed run

Streaming output truncated to the last 5000 lines.
Iteration:  72% 10460/14628 [45:33<17:53,  3.88it/s]
Iteration:  72% 10461/14628 [45:33<16:42,  4.16it/s]
Iteration:  72% 10462/14628 [45:33<16:03,  4.32it/s]
Iteration:  72% 10463/14628 [45:34<15:42,  4.42it/s]
Iteration:  72% 10464/14628 [45:34<15:29,  4.48it/s]
Iteration:  72% 10465/14628 [45:34<17:52,  3.88it/s]
Iteration:  72% 10466/14628 [45:34<16:37,  4.17it/s]
Iteration:  72% 10467/14628 [45:35<15:53,  4.36it/s]
Iteration:  72% 10468/14628 [45:35<15:35,  4.44it/s]
Iteration:  72% 10469/14628 [45:35<15:22,  4.51it/s]
Iteration:  72% 10470/14628 [45:35<17:49,  3.89it/s]
Iteration:  72% 10471/14628 [45:36<16:37,  4.17it/s]
Iteration:  72% 10472/14628 [45:36<15:57,  4.34it/s]
Iteration:  72% 10473/14628 [45:36<15:37,  4.43it/s]
Iteration:  72% 10474/14628 [45:36<15:25,  4.49it/s]
Iteration:  72% 10475/14628 [45:37<18:02,  3.84it/s]
Iteration:  72% 10476/14628 [45:37<16:45,  4.13it/s]
Iteration:  72% 10477/14628 [45:37<16:03,  4.31i

### Compute perplexity of a dataset.
This section shows how to compute perplexity of a dataset according to either the pre-trained or your fine-tuned language model. While this is possible to do by calling `run_language_modeling.py` on the command-line as above, we'll instead call the Python functions directly.

#### Look at what checkpoints are available
Run `ls` to look at what checkpoints saved been saved. You'll want to set `CHECKPOINT_PATH` below to one of these in order to evaluate the model weights saved in that checkpoint.

In [6]:
!ls '/content/drive/My Drive/finetuned_models/wodehouse'

checkpoint-1500   Final_model  pytorch_model.bin	training_args.bin
config.json	  merges.txt   special_tokens_map.json	vocab.json
eval_results.txt  output       tokenizer_config.json


#### Helper functions

In [7]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=None)

  model = AutoModelWithLMHead.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None
  )
  
  model.to(args.device)
  return model

def set_seed(seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)

  args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.evaluate(args, model, tokenizer, prefix="")
  return result

#### Compute it.

In [8]:
class DictToObject(object):

    def __init__(self, dictionary):
        def _traverse(key, element):
            if isinstance(element, dict):
                return key, DictToObject(element)
            else:
                return key, element

        objd = dict(_traverse(k, v) for k, v in dictionary.items())
        self.__dict__.update(objd)

In [9]:
# Set this to the checkpoint you want to evalute, or to "gpt2-medium" to
# evaluate the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/wodehouse/'
OUTPUT_PATH = '/content/drive/My Drive/finetuned_models/wodehouse/output'

# Set this to the list of text files you want to evaluate the perplexity of.
DATA_PATHS = ["/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/validate.txt",
              "/content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/test.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=OUTPUT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = DictToObject(args)

model = load_model(args)

# for data_path in DATA_PATHS:
#   eval_results = do_perplexity_eval(args, model, data_path)
#   perplexity = eval_results['perplexity']
#   print('{} is the perplexity of {} according to {}'.format(
#       perplexity, data_path, CHECKPOINT_PATH))

10/21/2020 21:14:06 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/wodehouse/config.json
10/21/2020 21:14:06 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

10/21/2020 21:14:06 - INFO - trans

Running on device:  cuda


10/21/2020 21:15:11 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/wodehouse/config.json
10/21/2020 21:15:11 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

10/21/2020 21:15:11 - INFO - trans

19.814990997314453 is the perplexity of /content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/validate.txt according to /content/drive/My Drive/finetuned_models/wodehouse/


Evaluating: 100%|██████████| 480/480 [00:28<00:00, 16.75it/s]
10/21/2020 21:15:58 - INFO - run_language_modeling -   ***** Eval results  *****
10/21/2020 21:15:58 - INFO - run_language_modeling -     perplexity = tensor(14.7571)


14.757075309753418 is the perplexity of /content/drive/My Drive/Colab Notebooks/wodehouse_generator/data/test.txt according to /content/drive/My Drive/finetuned_models/wodehouse/


### Generate samples
The following code generates text samples that are are continuations of a provided prompt.

In [10]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [16]:
# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.

def generate_wodehouse_samples(prompt):

  # You should try out other prompts as well as no prompt at all.
  PROMPT = prompt
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  n_gpu = torch.cuda.device_count()
  print("Running on device: ", device)

  args = collections.defaultdict(
    model_name_or_path=CHECKPOINT_PATH,
    output_dir=OUTPUT_PATH,
    n_gpu=n_gpu,
    mlm=False,
    device=device,
    model_type='gpt2',
    seed=42,
    stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
    temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
    k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
    p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
    repetition_penalty=None,
    length=900,  # Number of tokens to generate.
    num_return_sequences=3,  # Number of independently computed samples to generate.
  )
  args = DictToObject(dict(args))

  model = load_model(args)
  sequences = generate_samples(args, model, PROMPT)

  return sequences


In [17]:
sequences = generate_wodehouse_samples("Seated with his wife at breakfast on the veranda which overlooked the rolling lawns and leafy woods of his charming Sussex home, Geoffrey Windlebird, the great financier, was enjoying the morning sun to the full. ")

for idx, sequence in enumerate(sequences):
  print('\n====== GENERATION {} ======'.format(idx))
  print(sequence)

10/21/2020 21:17:54 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/wodehouse/config.json
10/21/2020 21:17:54 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

10/21/2020 21:17:54 - INFO - trans

Running on device:  cuda


10/21/2020 21:18:07 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/wodehouse/config.json
10/21/2020 21:18:07 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

10/21/2020 21:18:07 - INFO - trans


====== GENERATION 0 ======
Seated with his wife at breakfast on the veranda which overlooked the rolling lawns and leafy woods of his charming Sussex home, Geoffrey Windlebird, the great financier, was enjoying the morning sun to the full.  He was
being a pleasant fellow, and had not expected his visitor to have a
familiar name. The only thing which made him so different from the other
well-known figures in the New York financial establishment--and he felt
uncomfortable--was that he thought of him as an old friend. In his
habit of collecting things at random, Geoffrey always found in his
own collection one that was either a gold coin or a hundred dollars
in gold. He did not believe in collecting, but he was grateful that
he had this new toy in the form of himself. On the terrace he had a
stash of letters. A few were dated on the front page, and a few had pages
which looked to have been stamped on to them by hand. To Geoffrey, on reading a
dozen, there seemed to arise an impression tha

In [21]:
sequences = generate_wodehouse_samples("It was in Oxford Street at the hour when women come up from the suburbs to shop; and he was standing among the dogs and commissionaires outside Selfridge’s.")

for idx, sequence in enumerate(sequences):
  print('\n====== GENERATION {} ======'.format(idx))
  print(sequence)

10/21/2020 20:53:14 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/wodehouse/config.json
10/21/2020 20:53:14 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

10/21/2020 20:53:14 - INFO - trans

Running on device:  cuda


10/21/2020 20:53:54 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/wodehouse/config.json
10/21/2020 20:53:54 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

10/21/2020 20:53:54 - INFO - trans


====== GENERATION 0 ======
It was in Oxford Street at the hour when women come up from the suburbs to shop; and he was standing among the dogs and commissionaires outside Selfridge’s. He was
still in the habit of sitting up and listening to them speak quietly and
frequently, when he was offered to talk, and this year there had not
been a single dog in the neighbourhood. The nearest Mr Perceval
had came to talking was from Mr Perceval, a dog in a neighbourhood so
small and isolated--it consisted of only six streets in each--that it
may not even be possible for a man of his position to listen
peacefully.

He was aware that, for a moment, he was compelled to look over his shoulder, but
he had done so only to see a slight gurgle of what might have been
a sheep.

“That’s not worth the candle.”

“Why, what on earth is your name going to be?”

“Yes, yes! I’ve just come in to the office,” said Mr Perceval, starting
down the street.

Lord Emsworth’s head rose from his chair. His face was palli